In [1]:
# Imppporting and loading all modules
!pip install ipywidgets --quiet

import numpy as np
import joblib
from ipywidgets import Text, HTML, VBox, Button
from IPython.display import display
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load models
model_0 = joblib.load("nxt-model.pkl")   # next word model
tokenizer = joblib.load("tokenizer.pkl")    # tokenizer

spam_model = joblib.load("model.pkl")        # spam classifier
cv = joblib.load("cv.pkl")                   # count vectorizer


#Step 2: Next Word Prediction Function
def predict_next_word(input_text):
    seq = tokenizer.texts_to_sequences([input_text])[0]

    if len(seq) == 0:
        return "(no prediction)"

    padded = pad_sequences([seq], maxlen=24, padding='pre')
    pred = model_0.predict(padded, verbose=0)

    index = np.argmax(pred)

    for word, i in tokenizer.word_index.items():
        if i == index:
            return word

    return "(no prediction)"

#Step 3: UI Components

textbox = Text(
    placeholder="Type a message...",
    description="Text:",
    layout={'width': '600px'}
)

next_word_output = HTML(value="<b>Next Word:</b> ")
spam_output = HTML(value="<b>Spam Result:</b> ")

spam_button = Button(
    description="Check Spam",
    button_style='info'
)


#Step 4: Auto Next-Word Update

def on_text_change(change):
    text = change["new"]
    next_word = predict_next_word(text)
    next_word_output.value = f"<b>Next Word:</b> {next_word}"

textbox.observe(on_text_change, names='value')


# Step 5: Spam Check Button Logic


def check_spam(button):
    text = textbox.value

    if text.strip() == "":
        spam_output.value = "<b>Spam Result:</b> Please type text first"
        return

    text_vector = cv.transform([text])
    prediction = spam_model.predict(text_vector)

    if prediction[0] == 1:
        spam_output.value = "<b>Spam Result:</b> 🚫 Spam Message"
    else:
        spam_output.value = "<b>Spam Result:</b> ✅ Not Spam"

spam_button.on_click(check_spam)




# Step 6: Display Everything

display(VBox([
    textbox,
    next_word_output,
    spam_button,
    spam_output
]))




